# <<데이터분석과기계학습 2차 project 중간 보고서>>

## 주제 : 어린이 위험 행동 인식 후, 운전자에게 먼저 알러주는 서비스

1차에 발표했던 행동인식이 과연 이 주제의 목적에 적합한가에 대한 고민을 많이 해봤다.

그리고 내가 현재 직면한 하드웨어의 한계를 생각했을 때에도 어려움이 크다고 판단하여,

cctv 데이터를 활용하여 어린이와 어른을 구분하고

운전자에게 어린이 보호구역에 어린이가 존재한다는 것을 알리는 서비스로 추진하자고 약간의 변동을 줬다.






# *$데이터 수집*

aihub.or.kr 사이트에서 어린이 보호구역 내 어린이 도로보행 위험행동 영상의 데이터를 가져왔다.

이 데이터는 총 5.95TB의 용량을 가지고 있어, 일부 데이터인 하나의 폴더만 가져와서 분석하기로 결정했다.

https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=169

그리고 이 하나의 폴더도 용량이 커, 이 폴더 속에 있는 영상데이터들 중에 어린이가 많은 영상 20개를 추려 분석을을 진행할할 것이다.

In [ ]:
data_txt = open("./cctv_data/Training/training_dataset.txt",'r') #validation도 따로 진행함.
data = data_txt.read().splitlines()
result1 = []
for i in data:
    line = i.split()
    if ("./cctv_data/Training/label_data/"+line[0][:19]+'.json') in result1:
        continue
    else:
        result1.append("./cctv_data/Training/label_data/"+line[0][:19]+'.json')
print(len(result1))


303


# *$Data Processing*

### >303개의 영상과 json파일을 학습시키기에는 무리가 있다고 판단하였다.
303개의 영상을 frame단위로 캡처하게 된다면, 30만개 이상의 사진이 저장되기 때문에 무리가 있다.


그러기에 나는 주제에 관련된 데이터만 추출하기 위해


어린이가 존재하는 영상만 추출한 후, 전처리를 진행해야겠다고 판단함.

어린이가 많은 영상 20개를 추리기 위해 아래 processing_annotations함수를 영상의 label데이터들이 

있는 json 파일을 모두 열어서 one hot encoding 개념을 활용해서 하나의 txt파일에 저장했다.

train data는 training_dataset.txt

validation data는 validation_dataset.txt에 저장하여 활용할 수 있도록 했다.

이 txt파일에는 각 프레임별 캡처 사진에 대한 정보가 있는데,

"파일명.json 1 0 0 1 0 "... 이런식으로 각 해당하는 항목에 1을 넣어 희소행렬처럼 생겼다.

In [ ]:
#파일 이름을 간단하게 변경! (파일 날짜와 시간만 사용.)
import os


path = "./cctv_data/Training/label_data" #validation도 따로 진행함.

file_list = os.listdir(path)
file_list

for file in file_list: # 전체 파일 리스트에 대해서 수행
    print(file)
    src = os.path.join(path, file) # 기존 파일 경로
    dst_name = file[:19]+'.json' # 이름 수정 json 파일은 json을 추가 mp4는 mp4를 추가
    dst = os.path.join(path, dst_name) # 바뀐 이름으로 저장할 경로
    os.rename(src, dst) # rename 수행

2021-09-07_08-15-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-07_16-00-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-01_18-48-00_wed_sunny_out_ye-ma_CD0001.json
2021-09-09_20-18-00_thu_sunny_out_ye-ma_CD0002.json
2021-09-07_14-36-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-08_10-06-00_wed_sunny_out_do-sa_C0001-1_고1.json
2021-09-08_09-18-00_wed_sunny_out_ye-ma_CD0001.json
2021-09-07_22-21-00_tue_sunny_out_ye-ma_CD0002.json
2021-09-07_09-42-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-07_11-24-00_tue_cloudy_out_do-sa_C0053-1.json
2021-09-09_09-51-00_thu_sunny_out_do-sa_C0053-1.json
2021-09-02_09-09-00_thu_sunny_out_ye-ma_CD0001.json
2021-09-07_22-45-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-03_08-39-00_fri_cloudy_out_do-sa_C0053-1.json
2021-09-07_08-42-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-07_22-48-00_tue_sunny_out_ye-ma_CD0001.json
2021-09-06_17-18-00_mon_cloudy_out_do-sa_C0053-1.json
2021-09-08_09-27-00_wed_sunny_out_do-sa_C0001-1_고1.json
2021-09-09_09-33-00_thu_sunny_out_ye-ma_CD0001.js

In [ ]:
#용량 부족으로 인해 어린이가 존재하지 않는 영상 제거 하는 전처리 작업
data_txt = open("./cctv_data/Validation/validation_dataset.txt",'r') #train도 따로 진행함.
data = data_txt.read().splitlines()
result = {}
for i in data:
    line = i.split()
    if (line[3] == '1' or line[4] == "1"):
        
        if ("./cctv_data/Validation/label_data/"+line[0][:19]+'.json') in result.keys():
            result["./cctv_data/Validation/label_data/"+line[0][:19]+'.json'] += 1
            continue
        else:
            result["./cctv_data/Validation/label_data/"+line[0][:19]+'.json'] = 1
            # result.append("./cctv_data/Validation/label_data/"+line[0][:19]+'.json')

result_asc = sorted(result.items(), key = lambda item: item[1],reverse = True)
choice_file = []
for key, value in result_asc[:5]: #validation은 child가 가장 많은 상위 5개를 train은 20개를 choice_file 리스트에 넣었다.
    total = 0
    for d in data:
        b = key.split("/")[-1][:-5]
        if b in d:
            total += 1
    print(key, ":", value,"/",total)
    choice_file.append(key)
#result
            

./cctv_data/Validation/label_data/2021-09-06_08-12-00.json : 150 / 539
./cctv_data/Validation/label_data/2021-09-03_07-51-00.json : 98 / 803
./cctv_data/Validation/label_data/2021-09-05_16-24-00.json : 80 / 2484
./cctv_data/Validation/label_data/2021-09-07_21-45-00.json : 80 / 954
./cctv_data/Validation/label_data/2021-09-08_09-18-00.json : 73 / 3202


### --위 리스트 choice_file에서 선정된 파일만 놔두고 나머지 데이터들을 제거하자.

train은 20개 / validation은 5개 파일만 보존

In [ ]:
import os

path = "./cctv_data/Validation/label_data" #train도 따로 진행함.

data_list = os.listdir(path)
data_list
count = 0

for file in data_list:
    file_path = "./cctv_data/Validation/label_data/" + file
    if file_path in choice_file:
        #print("yes")
        if os.path.exists(file_path):
            print(file_path,"보존")
            count+=1
            continue
    else:
        if file_path in ['./cctv_data/Validation/label_data/.ipynb_checkpoints','./cctv_data/Validation/label_data/.ipynb_checkpoints.json']:
            continue
        elif os.path.exists(file_path):
            os.remove(file_path)
print(count)

./cctv_data/Validation/label_data/2021-09-05_16-24-00.json 보존
./cctv_data/Validation/label_data/2021-09-06_08-12-00.json 보존
./cctv_data/Validation/label_data/2021-09-03_07-51-00.json 보존
./cctv_data/Validation/label_data/2021-09-07_21-45-00.json 보존
./cctv_data/Validation/label_data/2021-09-08_09-18-00.json 보존
5


## *#class & bbox 좌표 데이터 구성*

데이터를 모델에 돌리기 전에 모델에 적합한 형태로 구성해줘야한다.

우선 수집한 데이터의 형태는 label dataset이 json파일로 각 프레임마다 어떤 객체가 어디 좌표에 있는지 어떤 옷을 입고 있는지, 나이대, 성별, 어디로 향하는지가 담겨있다.

여기서 내가 원하는 부분은 사람인 객체의 나이대와 bounding box 좌표다.

그렇기에, 아래 코드를 활용하여 json 파일에서 원하는 정보만 가져와 txt파일에 담았다.




In [ ]:
import cv2
import os
import json
from operator import itemgetter

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' +  directory)

def processing_annotations(json_path):
    with open(json_path, 'r', encoding='utf8') as json_f:
        json_data = json.load(json_f)

        categories = json_data["categories"]

        annotations = json_data["annotations"]
        sort_annotations = sorted(annotations, key=itemgetter("frame"))
        process_annotations = []
        for annotation in sort_annotations:
            if annotation["id"][0] == "p" or annotation["id"][0] == "a":
                annotation["age"] = [category["age"] for category in categories if
                                     category["id"] == annotation["id"]][0]
                process_annotations.append(annotation)

    return process_annotations

def extract_frame(video_path, process_annotations, file_name,total_frame):
    cap = cv2.VideoCapture(video_path)
    frame_no = 0
    checked_idx = 0
    cap_img = []
    if cap.isOpened():
        while True:
            ret, img = cap.read()
            if ret:
                split_process_annotations = process_annotations[checked_idx:]
                for process_annotation in split_process_annotations:

                    if process_annotation["frame"] > frame_no:
                        break
                    if process_annotation["frame"] % 10 == 0:
                        bbox = process_annotation["bbox"]
                        crop_img = img.copy()
                        jpg_file_name = file_name + "_" + str(process_annotation["frame"]) + ".jpg"
                        cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)

                        age_w = {"child": "1", "teenager": "1", "adult": "0", "senior": "0"}.get(process_annotation["age"])

                        dataset_line = age_w + " " + str(bbox[0])  + " " + str(bbox[1])  + " " + str(bbox[2]) + " " + str(bbox[3]) + "\n"
                        label_txt = "/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label/"+jpg_file_name[:-4]+".txt"

                        if jpg_file_name in cap_img:

                            with open(label_txt, 'a', encoding='utf8') as train_f:
                                  train_f.write(dataset_line)

                        else:
                            cv2.imwrite('/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/img/' + jpg_file_name, crop_img)
                            cap_img.append(jpg_file_name)
                            with open(label_txt, 'a', encoding='utf8') as train_f:
                                  train_f.write(dataset_line)

                    checked_idx += 1
                    
                frame_no += 1

            else:
                break
    else:
        print("can't open video.")
    cap.release()
    cv2.destroyAllWindows()

def processing(video_dir):
    for (root, dirs, files) in os.walk(video_dir):
        for file in files:

            file_name, extension = os.path.splitext(file)

            if extension == ".mp4":

                json_path = root[:-10] + "/"+'label_data/'+ file_name + ".json"
                with open(json_path, 'r', encoding='utf8') as json_f:
                    json_data = json.load(json_f)
                    video_impo = json_data["video"]
                    total_frame = video_impo["total_frame"]
                print(json_path)
                if os.path.exists(json_path):
                    process_annotations = processing_annotations(json_path)

                else:
                    with open("./error.txt", 'w', encoding='utf8') as error_f:
                        error_f.write(root + "/" + file + "\n")
                    print("Error : not matched " + root + "/" + file)
                    continue

                video_path = root + "/" + file
                extract_frame(video_path, process_annotations, file_name,total_frame)

if __name__ == "__main__":
    createFolder("/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/img")
    createFolder("/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label")

    video_dir = "/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/main_data"
    processing(video_dir)

/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-08_18-12-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-06_16-00-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-08_19-42-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-08_16-33-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-07_16-09-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-08_18-51-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-07_18-21-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/Training/label_data/2021-09-07_16-24-00.json
/content/drive/MyDrive/pytorch_advanced/objectdetection/data/cctv_data/T

# 결론!!

train img data는 933개

validation img data는 193개로 만들어졌다.